In [1]:
# 라이브러리 추가
import pandas as pd
import re

In [2]:
# 기사 제목
f = open("D:/GEODATA/bigkinds.txt", 'r', encoding='UTF-8')
lines = f.readlines()
f.close()

In [3]:
# 한나눔 불러오기
from konlpy.tag import Hannanum
hannanum = Hannanum()

In [4]:
# 단어 2차원 리스트
dataset = []
for i in range(len(lines)):
    dataset.append(hannanum.nouns(re.sub('[^가-힣a-zA-Z\s]', '', lines[i])))
dataset[:10]

[['구상나무', '산굴뚝나비', '한라산국립공원', '대표', '생물종'],
 ['한라산', '대표', '생물종', '구상나무', '산굴뚝나비', '선정'],
 ['한라산', '대표', '생물', '구상나무산굴뚝나비', '선정'],
 ['한라산', '생태', '특성', '대표', '생물종', '구상나무', '산굴뚝나비', '선정'],
 ['제주', '한라산', '대표', '생물종', '구상나무', '산굴뚝나비'],
 ['한라산국립공원', '대표', '생물'],
 ['장태산', '문재', '코스', '수', '생물친구들'],
 ['강릉제모습', '순포습지', '순채', '복원', '성', '미미'],
 ['쉬다가고파'],
 ['계룡산', '호반새']]

In [5]:
# 트랜잭션 인코더
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df.head(10)

,EM투입,LG디스플레,가게,가득,가속,가시연,가시연습지수생태계,가시화,가야산,가야산은분취,...,희생,흰꼬리수리,히어,히어리,히어리꽃,힐링,힐링관광,힐링하,힘,힘찬
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [6]:
# 어프라이어리(Apriori) 알고리즘
from mlxtend.frequent_patterns import apriori
frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

In [11]:
# 항목집합 길이가 2면서 지지도
frequent_itemsets[(frequent_itemsets['length'] == 2) &
                  (frequent_itemsets['support'] >= 0.01)].sort_values(by='support', ascending=False).head(10)

,support,itemsets,length
74,0.034951,"(선정, 깃대종)",2
73,0.020388,"(대전시, 깃대종)",2
77,0.020388,"(담비, 속리산)",2
82,0.020388,"(속리산, 서식)",2
76,0.016505,"(담비, 서식)",2
85,0.016505,"(확인, 서식)",2
87,0.013592,"(속리산, 하늘다람쥐)",2
72,0.013592,"(깃대종, 대전)",2
71,0.012621,"(호반새, 계룡산)",2
75,0.011650,"(종, 깃대종)",2


In [12]:
# 연관규칙(association rules) 분석
from mlxtend.frequent_patterns import association_rules
association_rules(frequent_itemsets, metric="lift",
                  min_threshold=1).sort_values(by='support', ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,(선정),(깃대종),0.048544,0.126214,0.034951,0.720000,5.704615,0.028825,3.120666
5,(깃대종),(선정),0.126214,0.048544,0.034951,0.276923,5.704615,0.028825,1.315844
41,(깃대종),(대전시),0.126214,0.034951,0.020388,0.161538,4.621795,0.015977,1.150975
40,(대전시),(깃대종),0.034951,0.126214,0.020388,0.583333,4.621795,0.015977,2.097087
31,(속리산),(담비),0.026214,0.022330,0.020388,0.777778,34.830918,0.019803,4.399515
8,(속리산),(서식),0.026214,0.040777,0.020388,0.777778,19.074074,0.019319,4.316505
9,(서식),(속리산),0.040777,0.026214,0.020388,0.500000,19.074074,0.019319,1.947573
30,(담비),(속리산),0.022330,0.026214,0.020388,0.913043,34.830918,0.019803,11.198544
16,"(담비, 속리산)",(서식),0.020388,0.040777,0.016505,0.809524,19.852608,0.015673,5.035922
37,(서식),(확인),0.040777,0.024272,0.016505,0.404762,16.676190,0.015515,1.639223
